In [ ]:
from IPython.display import display
from ipywidgets import VBox, HBox, Layout
import ipywidgets as widgets
from traitlets import Float

from one_locus_two_alleles_simulator import GenotypicFreqs


class GenoFreqsWidget(widgets.Box):
    freq_AA = Float().tag(sync=True)
    freq_Aa = Float().tag(sync=True)
    freq_aa = Float().tag(sync=True)

    def __init__(
        self,
        freq_AA: float,
        freq_Aa: float,
        freq_aa: float | None = None,
        *args,
        **kwargs
    ):
        if freq_aa is None:
            freq_aa = 1 - freq_AA - freq_Aa
        GenotypicFreqs(freq_AA, freq_Aa, freq_aa)

        self.freq_AA = freq_AA
        self.freq_Aa = freq_Aa
        self.freq_aa = freq_aa

        self.freqs_slider = widgets.FloatRangeSlider(
            value=(freq_AA, freq_AA + freq_Aa),
            min=0.0,
            max=1.0,
            step=0.01,
            readout=False,
        )
        self.freqs_slider.layout = Layout(width="auto", flex="1 1 auto")
        self.text_AA = widgets.FloatText(
            value=round(freq_AA, 2), description="Freq AA:", disabled=True
        )
        self.text_Aa = widgets.FloatText(
            value=round(freq_Aa, 2), description="Freq Aa:", disabled=True
        )
        self.text_aa = widgets.FloatText(
            value=round(freq_aa, 2), description="Freq aa:", disabled=True
        )

        self.freqs_slider.observe(self._update_slider, names="value")

        freqs_box = VBox(
            [
                HBox([self.text_AA, self.text_Aa, self.text_aa]),
                HBox([self.freqs_slider]),
            ]
        )
        super().__init__(children=[freqs_box], *args, **kwargs)

    def _update_slider(self, change):
        new_value = change["new"]
        self.freq_AA = new_value[0]
        self.freq_Aa = new_value[1] - self.freq_AA
        self.freq_aa = 1 - self.freq_AA - self.freq_Aa
        self.text_AA.value = round(self.freq_AA, 2)
        self.text_Aa.value = round(self.freq_Aa, 2)
        self.text_aa.value = round(self.freq_aa, 2)

    def _get_value(self):
        return self.range_slider.value

    def _set_value(self, value: tuple[float, float]):
        self.range_slider.value = value

    value = property(_get_value, _set_value)


class OneLocusSimApp:
    def __init__(self):
        self.app_title = "One locus simulator"

    def _generate_layout(self):
        geno_freqs_widget = GenoFreqsWidget(0.3, 0.3)
        return VBox([geno_freqs_widget])

    def _setup_event_handlers(self):
        # self.datagrid.observe(self.plot_stock, names='selections')
        pass

    def run_application(self):
        self._setup_event_handlers()
        display(self._generate_layout())


app = OneLocusSimApp()
app.run_application()